In [9]:
import pandas as pd
import os, re

os.chdir("C:\\Users\\Zjaffen1\\Desktop\\Projects\\data\\rasa messages")

In [10]:
def get_second_intent(intent_list):
    if len(intent_list) > 1:
        end = intent_list[1].find(",")
        return intent_list[1][6:end]

def get_second_confidence(intent_list):
    if len(intent_list) > 1:
        start = intent_list[1].find(",") + 13
        end = intent_list[1].find(")")
        return float(intent_list[1][start:end])


In [11]:
# target CSV and directory to save results to
csv = "TMOOS_jan21_con90.csv"
dir_name = "tmoos_secondary_intents"

In [12]:
messages_df = pd.read_csv(csv)
print(len(messages_df))

36448


In [13]:
# intents string to list
messages_df['intent_ranking'] = messages_df['intent_ranking'].apply(lambda x: re.findall('\(.*?\)', x))

In [14]:
messages_df['secondary_intent'] = messages_df['intent_ranking'].apply(get_second_intent)

In [15]:
messages_df['secondary_confidence'] = messages_df['intent_ranking'].apply(get_second_confidence)

In [16]:
intents = messages_df.secondary_intent.unique()
sec_intents_ranking = []
os.makedirs(dir_name, exist_ok=True)
os.chdir(dir_name)
os.makedirs('intent_texts', exist_ok=True)
for intent in intents:
    temp_df = messages_df[messages_df['secondary_intent'] == intent]
    total = len(temp_df)
    mean = temp_df['secondary_confidence'].mean()
    median = temp_df['secondary_confidence'].median()
    sec_intents_ranking.append((intent, total, mean, median))

    # for csvs with messages
    messages_loc = "intent_texts\\" + intent + "_messages.csv"
    temp_df = temp_df[['input_text','secondary_confidence']]
    temp_df = temp_df.sort_values(by='secondary_confidence', ascending=False)
    temp_df.to_csv(messages_loc, index=False, encoding='utf-8')

sec_intents_ranking.sort(key = lambda x: x[1], reverse=True)
s_intents_df = pd.DataFrame(sec_intents_ranking, columns=['Intent', 'Num Occurences', 'Avg Confidence', 'Median Confidence'])
ranked_name = 'ranked_' + dir_name + ".csv"
s_intents_df.to_csv(ranked_name, index=False, encoding='utf-8')
print("finished exporting secondary intents data")

finished exporting secondary intents data
